## Data loading

In [1]:
from clef.utils.data_loading import load_datasets
import os

root_path = '../../'
out_dir = './data-out/no-evidence'

train, dev = load_datasets(preprocess=True,
                           add_author_name=True,
                           add_author_bio=True,
                           root_path= root_path,)

# ensure out_dir directories exist for later
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    if not os.path.exists(os.path.join(out_dir, 'eval')):
        os.makedirs(os.path.join(out_dir, 'eval'))

loaded 96 training json lines and 32 dev json lines.


In [2]:
# we fake similarity scores for RQ2

import json
from copy import deepcopy

rq2_dataset = deepcopy(dev)

for i, item in enumerate(rq2_dataset):
    item['retrieved_evidence'] = []

    # for author_account, tweet_id, tweet_text in item['evidence']:
    #     item['retrieved_evidence'] += [[
    #         author_account, #'author_account'
    #         tweet_id, #'authority_tweet_id'
    #         tweet_text, #'doc_text'
    #         1, #'rank'
    #         1, #'score'
    #     ]]

    rq2_dataset[i] = item
print(json.dumps(rq2_dataset[0], indent=2))

{
  "id": "AuRED_142",
  "rumor": "Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD",
  "label": "REFUTES",
  "timeline": [
    [
      "https://twitter.com/LBpresidency",
      "1556600039211925504",
      "Today, the President of the Republic, General Michel Aoun, signed 9 laws that were previously approved by the House of Representatives. Details at the following link: https://t.co/wmrSaaEwDu"
    ],
    [
      "https://twitter.com/LBpresidency",
      "1556559119045332992",
      "President Aoun received the Minister of Foreign Affairs and Expatriates, Dr. Abdullah Bouhabib, and the Minister of Social Affairs, Hector Hajjar, and discussed with them developments related to the file of displaced Syrians in Lebanon https://t.co/QLQAJSKzs1"
    ],
    [
      "https://twitter.com/LBpresidency",
      "1556558220533157890",
      "President Aoun received the new British Ambassador to Lebanon, Hamish Cowell, and discusse

## custom openai inference

In [8]:
from typing import NamedTuple

class VerificationResult(NamedTuple):
    label: str
    score: float

import os
from openai import OpenAI
from openai.types.chat import ChatCompletion

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"), # This is the default and can be omitted
)

# system_message = 'You are a helpful assistant to verify statements. You need to decide if a claim is true ("SUPPORTS"), or false ("REFUTES") or if you do know ("NOT ENOUGH INFO"). No yapping.'
system_message = """
You are a helpful assistant.
You need to decide if a claim is most likely plausible ("SUPPORTS"), most likely wrong ("REFUTES"), or other ("NOT ENOUGH INFO"). 
YOU MUST USE YOUR OWN GENERAL KNOWLEDGE TO MAKE YOUR DECISION.
You must also provide a confidence score between 0 and 1, indicating how confident you are in your decision.
Format your answer in JSON format, like this: {"decision": ["SUPPORTS"|"REFUTES"|"NOT ENOUGH INFO"], "confidence": [0...1]}
Set your own temperature to 0.
No yapping.
""" 

def get_completion(input_message) -> ChatCompletion:
    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": input_message}
        ]
    )

    return completion
    
def inference_openai(statement: str, evidence: str):
    input_text = f'The statement: "{evidence}"\nThe claim: "{statement}"'

    valid_labels = [
        "REFUTES",
        "NOT ENOUGH INFO",
        "SUPPORTS"
    ]
    
    result = get_completion(input_text)

    try:
        answer = result.choices[0].message.content
    except:
        print(f'ERROR: could not unpack response string from openai model: {result}')
        return ("NOT ENOUGH INFO", 1.0)

    try:
        if answer:
            decision, confidence = json.loads(answer).values()
        else:
            print(f'ERROR: answer was empty, parsed from result: {result}')
    except ValueError:
        print(f'ERROR: could not json-parse response from openai model: {answer}')
        return ("NOT ENOUGH INFO", 1.0)

    if decision in valid_labels:
        return (decision, confidence)
    else:
        return ("NOT ENOUGH INFO", 1.0)

In [9]:
from typing import Callable, List
from tqdm.auto import tqdm
from clef.utils.data_loading import RankedDocs
import re

def factcheck_using_evidence(claim: str, evidence: List[RankedDocs], inference_method: Callable, debug: bool = False):
    predicted_evidence = []
    confidences = []

    if debug: tqdm.write(f'{claim}')

    label, confidence = inference_method(claim, '')

    if label == "REFUTES":
        # confidence is always positive, for REFUTES make confidence negative
        confidence *= -1
    elif label == "NOT ENOUGH INFO":
        confidence *= 0 # TODO uhmmm...

    if label != "NOT ENOUGH INFO":
        confidences += [confidence]

    if confidences:
        meanconf = sum(confidences) / len(confidences) # mean confidence, no weighting
    else:
        meanconf = 0
    
    if meanconf > 0.1:
        pred_label = "SUPPORTS"
    elif meanconf < -0.1:
        pred_label = "REFUTES"
    else:
        pred_label = "NOT ENOUGH INFO"
    
    return pred_label, predicted_evidence


from clef.utils.preprocessing import clean_tweet_aggressive
from tqdm.auto import tqdm


def check_dataset_with_model(dataset: List, model: str, debug: bool = False) -> List:
    res_jsons = []

    if model == 'bart':
        from clef.verification.models.bart import inference_bart
        inference_method = inference_bart
    elif model == 'roberta':
        from clef.verification.models.roberta import inference_roberta
        inference_method = inference_roberta
    elif model == 'openai':
        from clef.verification.models.openai import inference_openai
        inference_method = inference_openai
    else:
        print('[ERROR] invalid model string, available options: ["bart"|"roberta"|"openai"]')
        return []

    for item in tqdm(dataset):
        rumor = item["rumor"]
        retrieved_evidence = item["retrieved_evidence"]

        pred_label, pred_evidence = factcheck_using_evidence(rumor, retrieved_evidence, inference_method, debug)

        if debug:
            tqdm.write(f'label: {item["label"]}')
            tqdm.write(f'predicted: {pred_label}')
            tqdm.write('')
        
        res_jsons += [
            {
                "id": item["id"],
                "label": item["label"],
                "claim": rumor,
                "predicted_label": pred_label,
                "predicted_evidence": pred_evidence,
            }
        ]
    
    return res_jsons

### RQ2 openai

In [10]:
from clef.utils.data_loading import write_jsonlines_from_dicts

result = check_dataset_with_model(rq2_dataset, 'openai', debug=True)

outfile = f'{out_dir}/zeroshot-ver-rq2-openai.jsonl'
write_jsonlines_from_dicts(outfile, result)

  0%|          | 0/32 [00:00<?, ?it/s]

Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD
label: REFUTES
predicted: NOT ENOUGH INFO

In the video.. The spread of unidentified gunmen east of the capital, Baghdad https://t.co/L18KV8tKuZ
label: REFUTES
predicted: NOT ENOUGH INFO

*Riyad Salama to NBN, raising the value of the dollar through Resolution 158 from 8,000 LBP per dollar to 15,000 LBP, starting from the first of next month.* *Riyad Salama to NBN, we are raising the ceiling on banking withdrawals, starting from the first of next month, from $400 to $1,500 for individuals.*
label: REFUTES
predicted: NOT ENOUGH INFO

#Qatar threw Iranian peas into garbage for fear of the spread of the new #Corona_virus. https://t.co/1VEBDDOJ0I
label: REFUTES
predicted: NOT ENOUGH INFO

What Moza did towards the Yemeni people is a resounding scandal by preventing Yemenis from entering Qatar who hold the Haya card designated for visitors to Qatar for the World Cup season. Ho

## Evaluation

In [11]:
from clef.utils.scoring import eval_run_custom
from clef.utils.data_loading import task5_dir

import pandas as pd
from IPython.core.display import display_html

import datetime
from csv import writer

sample_submission_file = f'{root_path}/{task5_dir}/submission_samples/KGAT_zeroShot_verification_English_dev.json'

rq2_openai_submission_file = f'{out_dir}/zeroshot-ver-rq2-openai.jsonl'

ground_truth_file = f'{root_path}/{task5_dir}/data/English_dev.json'

eval RQ2

In [12]:
time_now  = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S') 
out_file = f'{out_dir}/eval/RQ2-{time_now}.csv'

eval_data = [
    ['baseline',    *eval_run_custom(sample_submission_file, ground_truth_file, out_file)],
    # ['RQ2-nl',      *eval_run_custom(rq2_nli_submission_file, ground_truth_file, out_file)],
    ['RQ2-openai',  *eval_run_custom(rq2_openai_submission_file, ground_truth_file, out_file)],
]

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ['method', 'macro-F1', 'strict-macro-F1']

df_r5  = eval_df[['method', 'macro-F1']].sort_values('macro-F1', axis=0, ascending=False)
df_map = eval_df[['method', 'strict-macro-F1']].sort_values('strict-macro-F1', axis=0, ascending=False)

df1_styler = df_r5.style.set_table_attributes("style='display:inline'").set_caption('macro-F1')
df2_styler = df_map.style.set_table_attributes("style='display:inline'").set_caption('strict-macro-F1')

eval_df.to_csv(out_file)

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

,method,macro-F1
0,baseline,0.508159
1,RQ2-openai,0.192593
,method,strict-macro-F1
0,baseline,0.508159
1,RQ2-openai,0.192593
